<a href="https://colab.research.google.com/github/Polavarapupavani26/PAVANI_23B21A4507_FMML_M1_L1/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1)

Yes, averaging the validation accuracy across multiple splits generally gives more consistent and reliable results. This is because:

1. Variance Reduction:
When you use a single validation split, the accuracy may fluctuate significantly depending on how well that specific split represents the overall data distribution. Some validation splits may by chance have more challenging or easier examples, leading to inconsistent results.
Averaging across multiple splits reduces the effect of random variability. Since you are sampling multiple times, the random variations in each split tend to cancel each other out, giving a more stable and reliable estimate of the model’s true performance.
2. Better Representation of Data:
By using multiple splits, the model gets to train and validate on different portions of the data. This gives a more comprehensive view of how well the model generalizes to unseen data.
For example, a difficult-to-predict subset of the data that might have been left out in one split will be included in another split, allowing for more robust performance evaluation.
3. Cross-Validation:
This approach is a simplified version of k-fold cross-validation, where the data is split into
𝑘
k subsets (or "folds"), and each fold serves as the validation set once, while the remaining
𝑘
−
1
k−1 folds are used as the training set. The results are averaged, providing a more generalized estimate of the model’s accuracy.
Averaging over more splits (or folds) generally leads to a more accurate approximation of how well the model would perform on truly unseen data.

2)


Yes, averaging validation accuracy across multiple splits can provide a more accurate estimate of test accuracy compared to using a single split for validation. Here's why:

1. Better Generalization Estimate:
When you use multiple splits, you expose the model to different subsets of the data for training and validation, which gives a broader view of how the model generalizes. This process reduces the bias caused by overfitting to a single train-validation split.
By taking an average across different splits, the result is a more reliable and robust estimate of how well the model will perform on unseen data (i.e., the test set).
2. Reduced Sensitivity to Data Splits:
A single validation set can give a misleading estimate if it happens to contain easier or harder examples than the test set. Multiple splits reduce this sensitivity by averaging the accuracy over various data partitions, making the validation performance more representative of the overall data distribution.
This prevents the model from being overly optimistic or pessimistic based on one specific split, bringing the validation accuracy closer to the true test accuracy.
3. Cross-Validation vs. Single Split:
In cross-validation (e.g., k-fold cross-validation), every data point gets a chance to be in both the training and validation set across different folds. This mimics the scenario where the model is trained on nearly all available data and tested on different subsets, giving a much more reliable estimate of the test performance.
For instance, if 5-fold cross-validation yields an average validation accuracy of 85%, it is likely that the test accuracy will be close to this value. In contrast, a single split might under- or overestimate test performance due to the randomness of that particular split.
4. Empirical Evidence:
In your lab, when you use multiple splits to calculate the average validation accuracy, it typically converges toward the test accuracy. For example, if your test accuracy is around 34.9%, you might find that the average validation accuracy from multiple splits is similar to this, while individual split accuracies may vary more widely.


3)


Yes, increasing the number of iterations generally leads to a better and more stable estimate of the model’s performance, including its generalization to unseen test data. Here’s why:

1. Variance Reduction with More Iterations:
Each iteration of train-validation splitting introduces randomness. With a small number of iterations, the estimate of the model's performance (like accuracy) can be highly influenced by this randomness.
More iterations help reduce variance by averaging out the effects of individual splits. As the number of iterations increases, the estimate becomes more consistent because the influence of outliers or particularly lucky/unlucky splits diminishes.
Key point: More iterations = more diverse splits = lower variance.

2. Diminishing Returns After a Certain Point:
While more iterations do reduce the variance in the estimate, the marginal improvement diminishes after a certain number of iterations. For instance, if you go from 2 to 10 iterations, you’ll notice a significant reduction in variance. But going from 10 to 50 iterations might not provide as much of an improvement.
Trade-off: More iterations require more computation time. So, it's important to balance the number of iterations with computational efficiency.
3. Better Sampling of Data:
More iterations provide the model with a more comprehensive view of the data by sampling different portions of the dataset for training and validation. This reduces the likelihood that your performance estimate is overly optimistic or pessimistic due to a specific split.
For example, if a small number of iterations happen to include a lot of “easy” samples in the validation set, the accuracy estimate could be unrealistically high. With more iterations, the model encounters a wider variety of data points, leading to a more realistic estimate.
4. Effect on Bias and Variance:
Bias: If the model is underperforming (e.g., due to poor hyperparameter choices or insufficient data), increasing the number of iterations will not fix that. More iterations don’t lower bias directly, but they help give a more precise picture of how biased the model is.
Variance: The larger the number of iterations, the lower the variance in your accuracy estimates. Lower variance means more reliable predictions and confidence in the model's true performance.


4)


Increasing the number of iterations does help improve estimates, but it cannot fully compensate for the limitations of a very small training or validation dataset. Here’s a breakdown of how increasing iterations interacts with small datasets:

1. Impact on Training with Small Datasets:
Small training datasets inherently limit the model’s ability to learn from the data. With limited data, the model is likely to underfit, resulting in poor generalization to unseen data.
Increasing the number of iterations in cross-validation will help get a more reliable estimate of performance across different train-validation splits, but it won’t improve the actual quality of the model’s training.
In other words, while more iterations will give a better sense of how poor the model might be performing due to limited training data, it won’t fix the fundamental problem of insufficient data for learning.
2. Impact on Validation with Small Datasets:
Small validation datasets increase the variability of the accuracy estimates. Each validation set might contain too few examples to represent the data distribution adequately, which leads to over-optimistic or over-pessimistic results.
Increasing iterations helps mitigate this variability by averaging the performance across multiple random splits. However, when the validation set is very small, even multiple iterations can’t overcome the lack of representative examples.
Thus, the accuracy might still swing significantly between iterations, and the confidence in the estimate will be lower.
3. Benefits of More Iterations:
For a small validation set, increasing the number of iterations will improve the estimate’s consistency, giving you a better idea of the model’s generalization ability across multiple splits. It will help smooth out the fluctuations caused by the small size of the validation set.
For a small training set, increasing iterations gives a more robust estimate of how well the model performs, but it won’t help improve the model’s performance itself. The quality of training still depends on the size and quality of the training data.
4. Diminishing Returns with Tiny Datasets:
With very small datasets, even increasing the number of iterations won’t yield much improvement because the model doesn’t have enough data to learn meaningful patterns, and the validation set is too small to provide reliable estimates. So, there’s a limit to how much iterations can help.
At some point, you will hit diminishing returns because the dataset size limits how well the model can perform.
5. Potential Overfitting in Small Training Sets:
With small training sets, there’s also a higher risk of overfitting—the model may memorize the few training examples, leading to high variance and poor performance on the test set.
Increasing iterations won’t solve the overfitting problem caused by insufficient training data.
6. Alternative Approaches:
Data augmentation: If your training data is too small, try generating more data through techniques like synthetic data generation, bootstrapping, or data augmentation.
Cross-validation: You can also try using more advanced cross-validation techniques like k-fold cross-validation to ensure that every data point is used in both training and validation at some point.
Smaller models: If data is too limited, consider using simpler models with fewer parameters to avoid overfitting.
Regularization: Use regularization techniques (like L1/L2 regularization) to reduce the risk of overfitting when working with small datasets.


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.